# Perceptron

In [ ]:
#xor-nn
import torch
X = torch.FloatTensor([[0,0],[0,1],[1,0],[1,1]])
Y = torch.FloatTensor([[0],[1],[1],[0]])
linear1 = torch.nn.Linear(2,2, bias=True)
linear2 = torch.nn.Linear(2,1, bias=True)
sigmoid = torch.nn.Sigmoid()
model = torch.nn.Sequential(linear1, sigmoid, linear2, sigmoid)
criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1)
for step in range(10001):
    optimizer.zero_grad()
    prediction = model(X)
    cost = criterion(prediction, Y)
    cost.backward()
    optimizer.step()
    if step%100 == 0:
        print(step, cost.item())

#MNIST

In [ ]:
#mnist-nn
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms

#hyperparameters
batch_size = 100
training_epochs = 15
learning_rate = 0.001

#load data
mnist_train = dsets.MNIST(root="MNIST_data/", train=True, transform=transforms.ToTensor(), download=True)
mnist_test = dsets.MNIST(root="MNIST_data/", train=False, transform=transforms.ToTensor(), download=True)
data_loader = torch.utils.data.DataLoader(dataset=mnist_train, batch_size=batch_size, shuffle=True, drop_last=True)

#nn layers
linear1 = torch.nn.Linear(784,256, bias=True)
linear2 = torch.nn.Linear(256,256, bias=True)
linear3 = torch.nn.Linear(256,10, bias=True)
relu = torch.nn.ReLU()

#weight initialization
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)

model = torch.nn.Sequential(linear1, relu, linear2, relu, linear3)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = len(data_loader)
    for X,Y in data_loader:
        X = X.view(-1,28*28)
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        avg_cost += cost/total_batch
    print("Epoch:{:04d} cost:{:.6f}".format(epoch+1, avg_cost))

KeyboardInterrupt: ignored

In [ ]:
#test
with torch.no_grad():
    X_test = mnist_test.test_data.view(-1,28*28).float()
    Y_test = mnist_test.test_labels

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction,1) == Y_test
    accuracy = correct_prediction.float().mean()
    print("Accuracy:",accuracy.item())

# Dropout

In [ ]:
#mnist-nn (+dropout)
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms

#hyperparameters
batch_size = 100
training_epochs = 15
learning_rate = 0.001

#load data
mnist_train = dsets.MNIST(root="MNIST_data/", train=True, transform=transforms.ToTensor(), download=True)
mnist_test = dsets.MNIST(root="MNIST_data/", train=False, transform=transforms.ToTensor(), download=True)
data_loader = torch.utils.data.DataLoader(dataset=mnist_train, batch_size=batch_size, shuffle=True, drop_last=True)

#nn layers
linear1 = torch.nn.Linear(784,256, bias=True)
linear2 = torch.nn.Linear(256,256, bias=True)
linear3 = torch.nn.Linear(256,10, bias=True)
relu = torch.nn.ReLU()
dropout = torch.nn.Dropout(p=0.5)

#weight initialization
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)

model = torch.nn.Sequential(linear1,relu,dropout, linear2,relu,dropout, linear3)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

model.train()   #set the model to train mode (dropout=True)
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = len(data_loader)
    for X,Y in data_loader:
        X = X.view(-1,28*28)
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        avg_cost += cost/total_batch
    print("Epoch:{:04d} cost:{:.6f}".format(epoch+1, avg_cost))

Epoch:0001 cost:0.442415
Epoch:0002 cost:0.211629
Epoch:0003 cost:0.168431
Epoch:0004 cost:0.142407
Epoch:0005 cost:0.132217
Epoch:0006 cost:0.115244
Epoch:0007 cost:0.109705
Epoch:0008 cost:0.103355
Epoch:0009 cost:0.098293
Epoch:0010 cost:0.092491
Epoch:0011 cost:0.088163
Epoch:0012 cost:0.087004
Epoch:0013 cost:0.081976
Epoch:0014 cost:0.077887
Epoch:0015 cost:0.078554


In [ ]:
#test (dropout off)
with torch.no_grad():
    model.eval()  #set the model to evalutaion mode (dropout=False)
    X_test = mnist_test.test_data.view(-1,28*28).float()
    Y_test = mnist_test.test_labels

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction,1) == Y_test
    accuracy = correct_prediction.float().mean()
    print("Accuracy:",accuracy.item())

Accuracy: 0.9782000184059143


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:57: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


# CNN

**Conv2d**\
torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True)

**Max-Pooling**\
torch.nn.MaxPool2d(kernel_size, stride=None, padding=0, dilation=1, return_indices=False, ceil_mode=False)

Shape check




In [ ]:
import torch
import torch.nn as nn
inputs = torch.Tensor(1,1,227,227)      #(batch_size, channel, height, width)
conv = nn.Conv2d(1,1,11, stride=4)
output = conv(inputs)
print(output.shape)

torch.Size([1, 1, 55, 55])


**MNIST CNN**

In [ ]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms

device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

#hyperparameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

#load data
mnist_train = dsets.MNIST(root="MNIST_data/", train=True, transform=transforms.ToTensor(), download=True)
mnist_test = dsets.MNIST(root="MNIST_data/", train=False, transform=transforms.ToTensor(), download=True)
dataloader = torch.utils.data.DataLoader(dataset=mnist_train, batch_size=batch_size, shuffle=True, drop_last=True)

In [ ]:
#Define CNN
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc1 = nn.Linear(3*3*128, 625, bias=True)
        self.fc2 = nn.Linear(625, 10)
        self.relu = nn.ReLU()
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [ ]:
model = CNN().to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
#training
total_batch = len(dataloader)
model.train()
for epoch in range(training_epochs):
    avg_cost = 0
    for X, Y in dataloader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        avg_cost += cost/total_batch
    print("Epoch:{} cost:{}".format(epoch+1, avg_cost))

In [ ]:
#testing
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction,1) == Y_test
    accuracy = correct_prediction.float().mean()
    print("Accuracy:", accuracy.item())

/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:57: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.9858999848365784
